# Baseline Models: Logistic Regression & Random Forest## 🎯 Concept PrimerBaselines sanity-check your preprocessing and provide a performance floor. If a neural net doesn't beat Logistic Regression, investigate why.**Model 1:** Logistic Regression (linear, interpretable)  **Model 2:** Random Forest (non-linear, feature importance)Expected: Train both models, evaluate on validation set, compare metrics.

## 📋 Objectives1. Train Logistic Regression with class weights2. Train Random Forest with hyperparameter tuning3. Evaluate both on validation set4. Compare metrics (Accuracy, Precision, Recall, F1, ROC-AUC)5. Visualize confusion matrices

## ✅ Acceptance Criteria- [ ] Logistic Regression trained and evaluated- [ ] Random Forest trained and evaluated- [ ] Metrics table comparing both models- [ ] Confusion matrices plotted- [ ] Best baseline identified

## 🔧 Setup

In [ ]:
# TODO 1: Import libraries# from sklearn.linear_model import LogisticRegression# from sklearn.ensemble import RandomForestClassifier# from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score# import matplotlib.pyplot as plt# import seaborn as sns

## 📊 Logistic Regression Baseline### TODO 2: Train Logistic Regression**Parameters:** Use class_weight='balanced' to handle imbalance  **Expected:** Fit on X_train, y_train; predict on X_val

In [ ]:
# TODO 2: Train Logistic Regression# lr = LogisticRegression(class_weight='balanced', random_state=42, max_iter=1000)# lr.fit(X_train, y_train)# y_pred_lr = lr.predict(X_val)# y_proba_lr = lr.predict_proba(X_val)[:, 1]

## 🌲 Random Forest Baseline### TODO 3: Train Random Forest**Parameters:** n_estimators=100, max_depth=10, class_weight='balanced'  **Expected:** Fit on X_train, y_train; predict on X_val

In [ ]:
# TODO 3: Train Random Forest# rf = RandomForestClassifier(n_estimators=100, max_depth=10, class_weight='balanced', random_state=42)# rf.fit(X_train, y_train)# y_pred_rf = rf.predict(X_val)# y_proba_rf = rf.predict_proba(X_val)[:, 1]

## 📈 Evaluate Baselines### TODO 4: Compute metrics for both models**Metrics:** Accuracy, Precision, Recall, F1, ROC-AUC  **Use:** classification_report and roc_auc_score

In [ ]:
# TODO 4: Evaluate models# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score# # metrics_lr = {#     'accuracy': accuracy_score(y_val, y_pred_lr),#     'precision': precision_score(y_val, y_pred_lr),#     'recall': recall_score(y_val, y_pred_lr),#     'f1': f1_score(y_val, y_pred_lr),#     'roc_auc': roc_auc_score(y_val, y_proba_lr)# }# # metrics_rf = {#     'accuracy': accuracy_score(y_val, y_pred_rf),#     'precision': precision_score(y_val, y_pred_rf),#     'recall': recall_score(y_val, y_pred_rf),#     'f1': f1_score(y_val, y_pred_rf),#     'roc_auc': roc_auc_score(y_val, y_proba_rf)# }# # print("Logistic Regression:", metrics_lr)# print("Random Forest:", metrics_rf)

## 🤔 Reflection1. Which baseline performs better? Why?2. What patterns do you see in confusion matrices?3. Are baselines good enough for your use case?4. What should the PyTorch model beat?

**Your reflection:***Write here*

## 📌 Summary✅ Baselines trained and evaluated  ✅ Metrics compared  ✅ Ready for PyTorch model**Next:** `07_pytorch_ffn_build_train.ipynb`